In [1]:
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

# assign to first GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#VRAM memory on-demand usage(prevent Keras takes all memory at the beginning)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   
sess = tf.Session(config=config)

KTF.set_session(sess)


Using TensorFlow backend.


In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
from keras import regularizers
import numpy as np

In [3]:
num_classes = 10
num_predictions = 20
batch_size = 64

(x_main, y_main), (x_test, y_test) = cifar10.load_data()

x_test = x_test.astype('float32')
y_test = keras.utils.to_categorical(y_test, num_classes)

In [4]:
def get_train_val(n,data,labels):#k=5
    val_start=n*10000
    val_end=val_start+10000
    x_val=data[val_start:val_end]
    y_val=labels[val_start:val_end]
    x_train=np.append(data[0:val_start],data[val_end:50000],axis=0)
    y_train=np.append(labels[0:val_start],labels[val_end:50000],axis=0)
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_val = keras.utils.to_categorical(y_val, num_classes)

    x_train = x_train.astype('float32')
    x_val=x_val.astype('float32')
    return x_train,y_train,x_val,y_val

In [5]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate



In [6]:
def train_model(n,x_train,y_train,x_val,y_val,x_test, y_test):
    weight_decay = 1e-4
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()
    
    #data augmentation
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        )
    datagen.fit(x_train)

    #training

    mc = ModelCheckpoint('k_fold_'+str(n)+'.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
    opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=150,\
                    verbose=1,validation_data=(x_val,y_val),callbacks=[LearningRateScheduler(lr_schedule),mc])
    scores = model.evaluate(x_test, y_test, batch_size=64, verbose=1)
    print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))


In [7]:
for i in range(0,5):
    x_train,y_train,x_val,y_val=get_train_val(i,x_main, y_main)
    train_model(i,x_train,y_train,x_val,y_val,x_test, y_test)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

625/625 [==============================] - 15s 23ms/step - loss: 0.8923 - acc: 0.7871 - val_loss: 0.8150 - val_acc: 0.7792

Epoch 00023: val_acc did not improve from 0.81440
Epoch 24/150
625/625 [==============================] - 15s 23ms/step - loss: 0.8857 - acc: 0.7908 - val_loss: 0.7655 - val_acc: 0.8021

Epoch 00024: val_acc did not improve from 0.81440
Epoch 25/150
625/625 [==============================] - 15s 23ms/step - loss: 0.8786 - acc: 0.7922 - val_loss: 0.9242 - val_acc: 0.7695

Epoch 00025: val_acc did not improve from 0.81440
Epoch 26/150
625/625 [==============================] - 15s 23ms/step - loss: 0.8660 - acc: 0.7941 - val_loss: 0.8570 - val_acc: 0.7854

Epoch 00026: val_acc did not improve from 0.81440
Epoch 27/150
625/625 [==============================] - 15s 24ms/step - loss: 0.8632 - acc: 0.7982 - val_loss: 0.7475 - val_acc: 0.8113

Epoch 00027: val_acc did not improve from 0.81440
Epoch 28/150
625/625 [==============================] - 15s 23ms/step - loss: 

625/625 [==============================] - 15s 24ms/step - loss: 0.7847 - acc: 0.8264 - val_loss: 0.7341 - val_acc: 0.8255

Epoch 00066: val_acc did not improve from 0.86050
Epoch 67/150
625/625 [==============================] - 15s 24ms/step - loss: 0.7828 - acc: 0.8299 - val_loss: 0.6726 - val_acc: 0.8415

Epoch 00067: val_acc did not improve from 0.86050
Epoch 68/150
625/625 [==============================] - 15s 24ms/step - loss: 0.7826 - acc: 0.8281 - val_loss: 0.6669 - val_acc: 0.8443

Epoch 00068: val_acc did not improve from 0.86050
Epoch 69/150
625/625 [==============================] - 15s 24ms/step - loss: 0.7656 - acc: 0.8321 - val_loss: 0.6422 - val_acc: 0.8473

Epoch 00069: val_acc did not improve from 0.86050
Epoch 70/150
625/625 [==============================] - 15s 24ms/step - loss: 0.7782 - acc: 0.8286 - val_loss: 0.8975 - val_acc: 0.7709

Epoch 00070: val_acc did not improve from 0.86050
Epoch 71/150
625/625 [==============================] - 15s 23ms/step - loss: 


Epoch 00108: val_acc did not improve from 0.88570
Epoch 109/150
625/625 [==============================] - 15s 24ms/step - loss: 0.5548 - acc: 0.8662 - val_loss: 0.5341 - val_acc: 0.8709

Epoch 00109: val_acc did not improve from 0.88570
Epoch 110/150
625/625 [==============================] - 15s 23ms/step - loss: 0.5587 - acc: 0.8651 - val_loss: 0.5979 - val_acc: 0.8585

Epoch 00110: val_acc did not improve from 0.88570
Epoch 111/150
625/625 [==============================] - 15s 24ms/step - loss: 0.5472 - acc: 0.8668 - val_loss: 0.5204 - val_acc: 0.8754

Epoch 00111: val_acc did not improve from 0.88570
Epoch 112/150
625/625 [==============================] - 15s 23ms/step - loss: 0.5422 - acc: 0.8664 - val_loss: 0.6394 - val_acc: 0.8414

Epoch 00112: val_acc did not improve from 0.88570
Epoch 113/150
625/625 [==============================] - 15s 24ms/step - loss: 0.5348 - acc: 0.8682 - val_loss: 0.5385 - val_acc: 0.8686

Epoch 00113: val_acc did not improve from 0.88570
Epoch 114

Epoch 1/150
625/625 [==============================] - 16s 26ms/step - loss: 2.0096 - acc: 0.4084 - val_loss: 1.5562 - val_acc: 0.5433

Epoch 00001: val_acc improved from -inf to 0.54330, saving model to k_fold_1.h5
Epoch 2/150
625/625 [==============================] - 15s 24ms/step - loss: 1.5171 - acc: 0.5493 - val_loss: 1.2942 - val_acc: 0.6059

Epoch 00002: val_acc improved from 0.54330 to 0.60590, saving model to k_fold_1.h5
Epoch 3/150
625/625 [==============================] - 15s 24ms/step - loss: 1.3478 - acc: 0.6144 - val_loss: 1.1845 - val_acc: 0.6393

Epoch 00003: val_acc improved from 0.60590 to 0.63930, saving model to k_fold_1.h5
Epoch 4/150
625/625 [==============================] - 15s 24ms/step - loss: 1.2365 - acc: 0.6514 - val_loss: 1.1666 - val_acc: 0.6636

Epoch 00004: val_acc improved from 0.63930 to 0.66360, saving model to k_fold_1.h5
Epoch 5/150
625/625 [==============================] - 15s 24ms/step - loss: 1.1645 - acc: 0.6785 - val_loss: 0.9451 - val_acc:

...

625/625 [==============================] - 14s 23ms/step - loss: 0.5320 - acc: 0.8683 - val_loss: 0.6414 - val_acc: 0.8459

Epoch 00129: val_acc did not improve from 0.87290
Epoch 130/150
625/625 [==============================] - 14s 23ms/step - loss: 0.5292 - acc: 0.8672 - val_loss: 0.5912 - val_acc: 0.8519

Epoch 00130: val_acc did not improve from 0.87290
Epoch 131/150
625/625 [==============================] - 15s 23ms/step - loss: 0.5238 - acc: 0.8696 - val_loss: 0.5473 - val_acc: 0.8703

Epoch 00131: val_acc did not improve from 0.87290
Epoch 132/150
625/625 [==============================] - 15s 23ms/step - loss: 0.5175 - acc: 0.8704 - val_loss: 0.5738 - val_acc: 0.8588

Epoch 00132: val_acc did not improve from 0.87290
Epoch 133/150
625/625 [==============================] - 15s 23ms/step - loss: 0.5142 - acc: 0.8701 - val_loss: 0.5071 - val_acc: 0.8791

Epoch 00133: val_acc improved from 0.87290 to 0.87910, saving model to k_fold_3.h5
Epoch 134/150
625/625 [==================

Epoch 2/150
625/625 [==============================] - 13s 21ms/step - loss: 1.4642 - acc: 0.5510 - val_loss: 1.2934 - val_acc: 0.5991

Epoch 00002: val_acc improved from 0.45840 to 0.59910, saving model to k_fold_4.h5
Epoch 3/150
625/625 [==============================] - 13s 21ms/step - loss: 1.2955 - acc: 0.6166 - val_loss: 1.5006 - val_acc: 0.5745

Epoch 00003: val_acc did not improve from 0.59910
Epoch 4/150
625/625 [==============================] - 13s 21ms/step - loss: 1.2768 - acc: 0.6411 - val_loss: 1.1548 - val_acc: 0.6702

Epoch 00004: val_acc improved from 0.59910 to 0.67020, saving model to k_fold_4.h5
Epoch 5/150
625/625 [==============================] - 13s 21ms/step - loss: 1.2297 - acc: 0.6619 - val_loss: 0.9655 - val_acc: 0.7006

Epoch 00005: val_acc improved from 0.67020 to 0.70060, saving model to k_fold_4.h5
Epoch 6/150
625/625 [==============================] - 13s 21ms/step - loss: 1.1386 - acc: 0.6837 - val_loss: 0.9610 - val_acc: 0.7026

Epoch 00006: val_acc 

625/625 [==============================] - 13s 21ms/step - loss: 0.6614 - acc: 0.8237 - val_loss: 0.6463 - val_acc: 0.8313

Epoch 00043: val_acc did not improve from 0.84030
Epoch 44/150
625/625 [==============================] - 13s 21ms/step - loss: 0.6601 - acc: 0.8254 - val_loss: 0.6766 - val_acc: 0.8273

Epoch 00044: val_acc did not improve from 0.84030
Epoch 45/150
625/625 [==============================] - 13s 21ms/step - loss: 0.6562 - acc: 0.8264 - val_loss: 0.6898 - val_acc: 0.8229

Epoch 00045: val_acc did not improve from 0.84030
Epoch 46/150
625/625 [==============================] - 13s 21ms/step - loss: 0.6502 - acc: 0.8265 - val_loss: 0.6423 - val_acc: 0.8380

Epoch 00046: val_acc did not improve from 0.84030
Epoch 47/150
625/625 [==============================] - 13s 21ms/step - loss: 0.6457 - acc: 0.8311 - val_loss: 0.7710 - val_acc: 0.8016

Epoch 00047: val_acc did not improve from 0.84030
Epoch 48/150
625/625 [==============================] - 13s 21ms/step - loss: 

625/625 [==============================] - 13s 21ms/step - loss: 0.4959 - acc: 0.8745 - val_loss: 0.5572 - val_acc: 0.8658

Epoch 00085: val_acc did not improve from 0.87720
Epoch 86/150
625/625 [==============================] - 13s 21ms/step - loss: 0.5013 - acc: 0.8730 - val_loss: 0.5400 - val_acc: 0.8677

Epoch 00086: val_acc did not improve from 0.87720
Epoch 87/150
625/625 [==============================] - 13s 21ms/step - loss: 0.4920 - acc: 0.8759 - val_loss: 0.4970 - val_acc: 0.8801

Epoch 00087: val_acc improved from 0.87720 to 0.88010, saving model to k_fold_4.h5
Epoch 88/150
625/625 [==============================] - 13s 21ms/step - loss: 0.4907 - acc: 0.8740 - val_loss: 0.5448 - val_acc: 0.8661

Epoch 00088: val_acc did not improve from 0.88010
Epoch 89/150
625/625 [==============================] - 13s 21ms/step - loss: 0.4899 - acc: 0.8741 - val_loss: 0.5564 - val_acc: 0.8606

Epoch 00089: val_acc did not improve from 0.88010
Epoch 90/150
625/625 [=======================

625/625 [==============================] - 13s 21ms/step - loss: 0.4547 - acc: 0.8819 - val_loss: 0.5764 - val_acc: 0.8570

Epoch 00128: val_acc did not improve from 0.88290
Epoch 129/150
625/625 [==============================] - 13s 21ms/step - loss: 0.4508 - acc: 0.8825 - val_loss: 0.5410 - val_acc: 0.8632

Epoch 00129: val_acc did not improve from 0.88290
Epoch 130/150
625/625 [==============================] - 13s 21ms/step - loss: 0.4512 - acc: 0.8813 - val_loss: 0.5482 - val_acc: 0.8616

Epoch 00130: val_acc did not improve from 0.88290
Epoch 131/150
625/625 [==============================] - 13s 21ms/step - loss: 0.4466 - acc: 0.8839 - val_loss: 0.4864 - val_acc: 0.8795

Epoch 00131: val_acc did not improve from 0.88290
Epoch 132/150
625/625 [==============================] - 13s 21ms/step - loss: 0.4455 - acc: 0.8819 - val_loss: 0.5422 - val_acc: 0.8565

Epoch 00132: val_acc did not improve from 0.88290
Epoch 133/150
625/625 [==============================] - 13s 21ms/step - l

In [14]:
res_acc=0.0
res_loss=0.0
for i in range(0,5):
    del model
    from keras.models import load_model
    model= load_model('k_fold_'+str(i)+'.h5')
    scores = model.evaluate(x_test, y_test, batch_size=64, verbose=1)
    print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))
    res_acc=res_acc+scores[1]*100
    res_loss=res_loss+scores[0]



10000/10000 [==============================] - 1s 121us/step

Test result: 88.120 loss: 0.488
10000/10000 [==============================] - 1s 128us/step

Test result: 88.220 loss: 0.466
10000/10000 [==============================] - 1s 133us/step

Test result: 87.940 loss: 0.489
10000/10000 [==============================] - 1s 141us/step

Test result: 87.540 loss: 0.531
10000/10000 [==============================] - 1s 146us/step

Test result: 88.080 loss: 0.471


In [15]:
print("AVG_Accuracy:",res_acc/5,',AVG_loss:',res_loss/5)

AVG_Accuracy: 87.97999999999999 ,AVG_loss: 0.48886562170982356
